In [2]:
import json
from pathlib import Path
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.schema import Document

In [109]:
from langchain_core.output_parsers import PydanticOutputParser
from src.utils.schemas import ArtIdeaSet
from typing import List, Optional, Literal, get_type_hints

parser = PydanticOutputParser(pydantic_object=ArtIdeaSet)

hints = get_type_hints(ArtIdeaSet)


In [116]:
from src.utils.schemas import ArtIdea


print(PydanticOutputParser(pydantic_object=ArtIdea).get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"id": {"title": "Id", "type": "string"}, "title": {"title": "Title", "type": "string"}, "drawing_prompt": {"title": "Drawing Prompt", "type": "string"}, "style_direction": {"title": "Style Direction", "type": "string"}, "why_it_fits_you": {"title": "Why It Fits You", "type": "string"}, "recommended_format": {"title": "Recommended Format", "type": "string"}, "difficulty": {"title": "Difficulty", "type": "string"}}, "required": ["id", "title", "drawing_prompt", "style_direction", "why_it_fits_you", "recommended_format", "difficul

In [143]:
abc = pydantic_schema_instructions(ArtIdeaSet)

{'$defs': {'ArtIdea': {'properties': {'id': {'title': 'Id', 'type': 'string'}, 'title': {'title': 'Title', 'type': 'string'}, 'drawing_prompt': {'title': 'Drawing Prompt', 'type': 'string'}, 'style_direction': {'title': 'Style Direction', 'type': 'string'}, 'why_it_fits_you': {'title': 'Why It Fits You', 'type': 'string'}, 'recommended_format': {'title': 'Recommended Format', 'type': 'string'}, 'difficulty': {'title': 'Difficulty', 'type': 'string'}}, 'required': ['id', 'title', 'drawing_prompt', 'style_direction', 'why_it_fits_you', 'recommended_format', 'difficulty'], 'title': 'ArtIdea', 'type': 'object'}}, 'properties': {'mood_or_focus': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Mood Or Focus'}, 'ideas': {'items': {'$ref': '#/definitions/ArtIdea'}, 'title': 'Ideas', 'type': 'array'}}, 'required': ['ideas'], 'title': 'ArtIdeaSet', 'type': 'object'}


In [157]:
def get_format_instructions():
    format_instructions = '''
    Respond ONLY with a JSON object matching this structure:
    {
    "mood_or_focus": "string or null",
    "ideas": [
        {
        "id": "idea_1",
        "title": "short catchy title",
        "drawing_prompt": "detailed prompt of what to draw",
        "style_direction": "how to style it (mood, colors, composition)",
        "why_it_fits_you": "why this idea matches the artist's style/history",
        "recommended_format": "reel" | "image post" | "corousel post",
        "difficulty": "easy" | "medium" | "hard"
        },
        ...
    ]
    }
    RULES:
- If you are running low on tokens, REDUCE the number of ideas automatically.
- Always prefer FEWER ideas over incomplete JSON.
- Every idea MUST include ALL fields exactly.
- No extra fields.
- Output VALID JSON only.
    '''
    return format_instructions

In [175]:
from langchain_core.prompts import PromptTemplate

# from src.utils.schemas import get_format_instructions

SYSTEM_PROMPT = """
You are an assistant helping a digital artist plan new artwork and Instagram content.
Use the context to suggest fresh ideas matching their style.
Generate a JSON object according to the format instructions. Do not include any explanations, backticks, or schemas.
{format_instructions}
"""

USER_PROMPT = """
You have access to this context about the artist:
{context}
The artist says: "{user_hint}"
Produce only complete ideas.
Never output a cut-off or incomplete idea.
For example, If 5 ideas won't fit within the limit, output 4 instead. If 4 won’t fit, output 3, and so on.
Now proceed to generate {num_ideas} ideas that would be exciting for this artist to draw and post on Instagram.
"""

prompt = PromptTemplate(
    input_variables=['context', 'user_hint', 'num_ideas'],
    partial_variables={"format_instructions": get_format_instructions()},
    template= f'{SYSTEM_PROMPT} \n {USER_PROMPT}'
)


In [176]:
print(prompt)

input_variables=['context', 'num_ideas', 'user_hint'] input_types={} partial_variables={'format_instructions': '\n    Respond ONLY with a JSON object matching this structure:\n    {\n    "mood_or_focus": "string or null",\n    "ideas": [\n        {\n        "id": "idea_1",\n        "title": "short catchy title",\n        "drawing_prompt": "detailed prompt of what to draw",\n        "style_direction": "how to style it (mood, colors, composition)",\n        "why_it_fits_you": "why this idea matches the artist\'s style/history",\n        "recommended_format": "reel" | "image post" | "corousel post",\n        "difficulty": "easy" | "medium" | "hard"\n        },\n        ...\n    ]\n    }\n    RULES:\n- If you are running low on tokens, REDUCE the number of ideas automatically.\n- Always prefer FEWER ideas over incomplete JSON.\n- Every idea MUST include ALL fields exactly.\n- No extra fields.\n- Output VALID JSON only.\n    '} template='\nYou are an assistant helping a digital artist plan 

In [177]:
from src.graph.ideation_module import get_style_context
from src.rag.llm import get_llm

context = get_style_context()

messages = prompt.format(
    context=context,
    user_hint="no specific hint",
    num_ideas='5',
)
model = get_llm()
response = model.invoke(messages)



In [178]:
print(response.content)

{
    "mood_or_focus": "emotional character design",
    "ideas": [
        {
            "id": "idea_1",
            "title": "Unsettling Family Portrait",
            "drawing_prompt": "Illustrate a family with dark, moody lighting and distorted features.",
            "style_direction": "monochrome with bold, expressive lines and heavy shading.",
            "why_it_fits_you": "This style fits the artist's preference for darker moods and emotional expressions.",
            "recommended_format": "image post",
            "difficulty": "hard"
        },
        {
            "id": "idea_2",
            "title": "Psychological Portrait of a Mindreader",
            "drawing_prompt": "Describe a character who is a master of mental manipulation and psychological control.",
            "style_direction": "monochrome with a focus on the character's eyes and facial expressions.",
            "why_it_fits_you": "This style aligns with the artist's love for emotional characters and dark mood

In [4]:
from src.utils.schemas import CaptionSet
from langchain_core.output_parsers import PydanticOutputParser


parser = PydanticOutputParser(pydantic_object=CaptionSet)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"idea_id": {"description": "Id of the idea", "title": "Idea Id", "type": "string"}, "captions": {"description": "List of captions for reel/post", "items": {"type": "string"}, "title": "Captions", "type": "array"}, "hashtags": {"description": "List of hastags to be included", "items": {"type": "string"}, "title": "Hashtags", "type": "array"}, "timelapse_tips": {"anyOf": [{"items": {"type": "string"}, "type": "array"}, {"type": "null"}], "default": null, "title": "Timelapse Tips"}}, "required": ["idea_id", "captions", "hashtags"]

In [ ]:
chain = template | model | parser
    response = chain.invoke()
    raw = response.content